In [1]:
import datetime
from statistics import mean
from pprint import pprint

import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from data import tfrecord_dataset, train_test_split
from train import best_num_epochs, train_model
from layers import SeluConv3D, SeluDense
from plot import plot_slice, plot_volume_animation
from config import (
    LIDC_SMALL_NEG_TFRECORD,
    LIDC_SMALL_POS_TFRECORD,
    SMALL_PATCH_SHAPE,
    BIG_PATCH_SHAPE,
    SEED,
)

%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 7]

In [2]:
# Hyperparameters
val_perc = 0.2
batch_size = 16
patience = 30
extra_epochs = 5
learning_rate = 1e-5

In [3]:
neg_x = tfrecord_dataset(LIDC_SMALL_NEG_TFRECORD)
num_neg_samples = sum(1 for _ in neg_x)
print(f"Number of negative samples: {num_neg_samples}")
neg_x

Number of negative samples: 375


<ParallelMapDataset shapes: (None, None, None, None), types: tf.float32>

In [4]:
neg_dataset = tf.data.Dataset.zip((neg_x, neg_x))
assert sum(1 for _ in neg_dataset) == num_neg_samples
neg_dataset

<ZipDataset shapes: ((None, None, None, None), (None, None, None, None)), types: (tf.float32, tf.float32)>

In [5]:
pos_x = tfrecord_dataset(LIDC_SMALL_POS_TFRECORD)
num_pos_samples = sum(1 for _ in pos_x)
print(f"Number of posative samples: {num_pos_samples}")
pos_x

Number of posative samples: 379


<ParallelMapDataset shapes: (None, None, None, None), types: tf.float32>

In [6]:
pos_dataset = tf.data.Dataset.zip((pos_x, pos_x))
assert sum(1 for _ in pos_dataset) == num_pos_samples
pos_dataset

<ZipDataset shapes: ((None, None, None, None), (None, None, None, None)), types: (tf.float32, tf.float32)>

In [7]:
dataset = neg_dataset.concatenate(pos_dataset)
num_samples = num_neg_samples + num_pos_samples
assert sum(1 for _ in dataset) == num_samples
dataset

<ConcatenateDataset shapes: ((None, None, None, None), (None, None, None, None)), types: (tf.float32, tf.float32)>

In [8]:
def build_model():
    encoder = keras.Sequential(
        [
            keras.layers.InputLayer(SMALL_PATCH_SHAPE, name="encoder_input"),
            SeluConv3D(filters=32, kernel_size=3, name="encoder_selu_conv3d_1"),
            keras.layers.MaxPooling3D((1, 2, 2), name="maxpool_1"),
            SeluConv3D(filters=64, kernel_size=3, name="encoder_selu_conv3d_2"),
            keras.layers.MaxPooling3D((1, 2, 2), name="maxpool_2"),
            SeluConv3D(filters=128, kernel_size=3, name="encoder_selu_conv3d_3"),
            keras.layers.MaxPooling3D((1, 2, 2), name="maxpool_3"),
            SeluConv3D(filters=256, kernel_size=3, name="encoder_selu_conv3d_4"),
            keras.layers.MaxPooling3D((1, 2, 2), name="maxpool_4"),
        ],
        name="encoder",
    )
    decoder = keras.Sequential(
        [
            keras.layers.InputLayer(encoder.output_shape[1:], name="decoder_input"),
            keras.layers.UpSampling3D((1, 2, 2), name="upsampling_1"),
            SeluConv3D(filters=256, kernel_size=3, name="decoder_selu_conv3d_1"),
            keras.layers.UpSampling3D((1, 2, 2), name="upsampling_2"),
            SeluConv3D(filters=128, kernel_size=3, name="decoder_selu_conv3d_2"),
            keras.layers.UpSampling3D((1, 2, 2), name="upsampling_3"),
            SeluConv3D(filters=64, kernel_size=3, name="decoder_selu_conv3d_3"),
            keras.layers.UpSampling3D((1, 2, 2), name="upsampling_4"),
            SeluConv3D(filters=32, kernel_size=3, name="decoder_selu_conv3d_4"),
            keras.layers.Dense(1, activation="sigmoid", name="decoder_final_dense"),
        ],
        name="decoder",
    )

    autoencoder = keras.Sequential(
        [
            keras.layers.InputLayer(SMALL_PATCH_SHAPE, name="autoencoder_input"),
            encoder,
            decoder,
        ],
        name="autoencoder",
    )
    return autoencoder

In [9]:
def compile_model(model):
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.MeanSquaredError(),
    )

In [10]:
train_dataset, val_dataset = train_test_split(
    dataset,
    test_perc=val_perc,
    cardinality=num_samples,
)
val_dataset = val_dataset.batch(1).cache().prefetch(tf.data.experimental.AUTOTUNE)
train_dataset = (
    train_dataset.batch(batch_size)
    .cache()  # must be called before shuffle
    .shuffle(buffer_size=64, reshuffle_each_iteration=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

autoencoder = build_model()
compile_model(autoencoder)

log_dir = f"logs/autoencoder-lidc"

num_epochs = best_num_epochs(
    autoencoder,
    train_dataset,
    val_dataset,
    patience,
    "val_loss",
    log_dir,
    verbose_training=1,
    metric_mode="min",
)
f"{num_epochs = }"

Epoch 1/1000
38/38 [==============================] - 2s 64ms/step - loss: 0.0925 - val_loss: 0.0556
Epoch 2/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0611 - val_loss: 0.0450
Epoch 3/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0551 - val_loss: 0.0419
Epoch 4/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0529 - val_loss: 0.0402
Epoch 5/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0517 - val_loss: 0.0394
Epoch 6/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0508 - val_loss: 0.0386
Epoch 7/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0503 - val_loss: 0.0379
Epoch 8/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0495 - val_loss: 0.0373
Epoch 9/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0490 - val_loss: 0.0368
Epoch 10/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0486 - val_lo

38/38 [==============================] - 2s 48ms/step - loss: 0.0427 - val_loss: 0.0310
Epoch 82/1000
38/38 [==============================] - 2s 47ms/step - loss: 0.0425 - val_loss: 0.0309
Epoch 83/1000
38/38 [==============================] - 2s 47ms/step - loss: 0.0425 - val_loss: 0.0309
Epoch 84/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0425 - val_loss: 0.0309
Epoch 85/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0425 - val_loss: 0.0309
Epoch 86/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0425 - val_loss: 0.0310
Epoch 87/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0425 - val_loss: 0.0316
Epoch 88/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0429 - val_loss: 0.0315
Epoch 89/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0426 - val_loss: 0.0309
Epoch 90/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0423 - val_loss: 0

Epoch 161/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0410 - val_loss: 0.0298
Epoch 162/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0410 - val_loss: 0.0298
Epoch 163/1000
38/38 [==============================] - 2s 47ms/step - loss: 0.0410 - val_loss: 0.0298
Epoch 164/1000
38/38 [==============================] - 2s 47ms/step - loss: 0.0410 - val_loss: 0.0298
Epoch 165/1000
38/38 [==============================] - 2s 47ms/step - loss: 0.0410 - val_loss: 0.0298
Epoch 166/1000
38/38 [==============================] - 2s 47ms/step - loss: 0.0410 - val_loss: 0.0298
Epoch 167/1000
38/38 [==============================] - 2s 47ms/step - loss: 0.0409 - val_loss: 0.0298
Epoch 168/1000
38/38 [==============================] - 2s 47ms/step - loss: 0.0409 - val_loss: 0.0297
Epoch 169/1000
38/38 [==============================] - 2s 47ms/step - loss: 0.0410 - val_loss: 0.0298
Epoch 170/1000
38/38 [==============================] - 2s 48ms/step - lo

38/38 [==============================] - 2s 48ms/step - loss: 0.0402 - val_loss: 0.0293
Epoch 241/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0402 - val_loss: 0.0294
Epoch 242/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0402 - val_loss: 0.0293
Epoch 243/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0402 - val_loss: 0.0293
Epoch 244/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0402 - val_loss: 0.0293
Epoch 245/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0402 - val_loss: 0.0293
Epoch 246/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0402 - val_loss: 0.0293
Epoch 247/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0402 - val_loss: 0.0294
Epoch 248/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0401 - val_loss: 0.0293
Epoch 249/1000
38/38 [==============================] - 2s 47ms/step - loss: 0.0401 - va

38/38 [==============================] - 2s 47ms/step - loss: 0.0398 - val_loss: 0.0292
Epoch 320/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0399 - val_loss: 0.0292
Epoch 321/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0397 - val_loss: 0.0292
Epoch 322/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0398 - val_loss: 0.0291
Epoch 323/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0397 - val_loss: 0.0292
Epoch 324/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0398 - val_loss: 0.0292
Epoch 325/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0397 - val_loss: 0.0291
Epoch 326/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0397 - val_loss: 0.0291
Epoch 327/1000
38/38 [==============================] - 2s 48ms/step - loss: 0.0397 - val_loss: 0.0293
Epoch 328/1000
38/38 [==============================] - 2s 47ms/step - loss: 0.0397 - va

38/38 [==============================] - 2s 47ms/step - loss: 0.0395 - val_loss: 0.0293
Epoch 399/1000
38/38 [==============================] - 2s 47ms/step - loss: 0.0395 - val_loss: 0.0293
Epoch 400/1000
38/38 [==============================] - 2s 47ms/step - loss: 0.0396 - val_loss: 0.0290
Epoch 401/1000
38/38 [==============================] - 2s 47ms/step - loss: 0.0394 - val_loss: 0.0290
Epoch 402/1000
38/38 [==============================] - 2s 47ms/step - loss: 0.0393 - val_loss: 0.0289
Epoch 403/1000
38/38 [==============================] - 2s 47ms/step - loss: 0.0393 - val_loss: 0.0290
Epoch 404/1000
38/38 [==============================] - 2s 47ms/step - loss: 0.0393 - val_loss: 0.0290
Epoch 405/1000
38/38 [==============================] - 2s 47ms/step - loss: 0.0393 - val_loss: 0.0290
Epoch 406/1000
38/38 [==============================] - 2s 47ms/step - loss: 0.0393 - val_loss: 0.0290
Epoch 407/1000
38/38 [==============================] - 2s 47ms/step - loss: 0.0393 - va

'num_epochs = 401'

In [11]:
train_dataset = (
    dataset.batch(batch_size)
    .cache()  # must be called before shuffle
    .shuffle(buffer_size=64, reshuffle_each_iteration=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
autoencoder = build_model()
compile_model(autoencoder)
model_fname = f"models/autoencoder-lidc.h5"
autoencoder = train_model(
    autoencoder,
    train_dataset,
    num_epochs + extra_epochs,
    model_fname,
)

48


In [ ]:
original, _ = next(iter(val_dataset.unbatch().batch(1).skip(6)))
encoder_out = autoencoder.get_layer("encoder")(original, training=False)
decoder_out = autoencoder.get_layer("decoder")(encoder_out, training=False)
plot_volume_animation(original[0, :])

In [ ]:
batch_index = 0
z_index = 5
fig, ax = plt.subplots(ncols=3)
plot_slice(original[batch_index, :], index=z_index, ax=ax[0])
plot_slice(encoder_out[batch_index, :], index=z_index, ax=ax[1])
plot_slice(decoder_out[batch_index, :], index=z_index, ax=ax[2])

In [ ]:
plot_animated_volume(original[0, :])